#### 可迭代对象

In [1]:
iterables = [
    "123",                  # string
    [1,2,3],                # list
    (1,2,3),                # tuple
    {1:"a", 2:"b", 3:"c"},  # dictionary keys  
    {1,2,3},                # set
]

for iterable in iterables:
    print(type(iterable))

    for item in iterable:
        print(item, end=',')
    
    print(' ')  # Blank line for better readability between different iterables

<class 'str'>
1,2,3, 
<class 'list'>
1,2,3, 
<class 'tuple'>
1,2,3, 
<class 'dict'>
1,2,3, 
<class 'set'>
1,2,3, 


#### 可迭代对象的接口

In [2]:
def common_attrs(*objs):
    """Return a set of common attributes of all objects in objs."""
    if not objs:
        return set()
    
    common = set(dir(objs[0]))
    for obj in objs[1:]:
        common &= set(dir(obj))
    common -= set(dir(object))  # Remove attributes from the base 'object' class
    return common

# 内省能力 内置的dir函数

In [3]:
iterable_common_attrs = common_attrs(*iterables)

print("Common attributes:", iterable_common_attrs)

# __contains__, __len__ 容器类型的可迭代对象

Common attributes: {'__contains__', '__len__', '__iter__'}


In [ ]:
# 文件对象也是可迭代对象
f = open("Sec1.ipynb", "r")

iterables.append(f)

iterable_common_attrs = common_attrs(*iterables)
print("Common attributes after adding file object:", iterable_common_attrs)

Common attributes after adding file object: {'__iter__'}


In [5]:
# 双下划线内置函数

# __iter__  -> 内置函数 iter()

for iterable in iterables:
    print(iter(iterable))  # 返回一个迭代器对象

<_io.TextIOWrapper name='1.ipynb' mode='r' encoding='UTF-8'>


#### 迭代器的接口

In [6]:
iterators = [iter(it) for it in iterables]

iterator_common_attrs = common_attrs(*iterators)

print("Common attributes of iterators:", iterator_common_attrs)
# 迭代器有两个接口： __iter__, __next__

Common attributes of iterators: {'__next__', '__iter__'}


#### 迭代器的用法

In [7]:
actions = ["wakeup", "brush", "wash", "eat", "sleep"] # 构建一个可迭代对象

action_iter = iter(actions) # 通过可迭代对象的__iter__方法，获取迭代器对象

In [8]:
# 迭代器的遍历
while True:
    try:
        action = next(action_iter) # 通过迭代器的__next__方法，获取下一个对象
        print(action)
    except StopIteration:  # 捕获迭代器结束的异常
        break


wakeup
brush
wash
eat
sleep


#### 自定义迭代器
> 基本功能：
>- 初始化时要传入可迭代对象  
>- 要初始化迭代进度  
>- 每次迭代时，即每次调用__next__方法时：
>>* 如果仍有元素可供迭代，则返回本轮迭代的对象，同时更新当前迭代进度；  
>>* 如果没有元素用来返回，则迭代结束，抛出StopIteration异常。
>
> 额外：
>- 设置黑名单，如果当前元素在黑名单，则跳过；  
>- 将符合条件的元素*2后返回。

In [9]:
BLACK_list = ['kill', 'shot']

class SelfDefineIter:
    def __init__(self, actions):
        self.actions = actions
        self.index = 0  # 记录当前迭代到的位置
    
    def __next__(self):
        while self.index < len(self.actions):
            action = self.actions[self.index]
            self.index += 1
            if action in BLACK_list:
                continue  # 跳过黑名单中的动作
            return action
        raise StopIteration  # 如果没有更多元素，抛出StopIteration异常

actions = ["wakeup", "brush", "wash", "eat", "sleep", "kill", "shot"]
custom_iter = SelfDefineIter(actions)
while True:
    try:
        action = next(custom_iter)
        print(action)
    except StopIteration:
        break

wakeup
brush
wash
eat
sleep


In [10]:
# for 循环需要的是【可迭代对象】，而不是【迭代器对象】
for x in custom_iter:  # 迭代器对象也可以直接用于for循环
    print(x)

TypeError: 'SelfDefineIter' object is not iterable

In [11]:
BLACK_list = ['kill', 'shot']

class SelfDefineIter:
    def __init__(self, actions):
        self.actions = actions
        self.index = 0  # 记录当前迭代到的位置
    
    def __next__(self):
        while self.index < len(self.actions):
            action = self.actions[self.index]
            self.index += 1
            if action in BLACK_list:
                continue  # 跳过黑名单中的动作
            return action
        raise StopIteration  # 如果没有更多元素，抛出StopIteration异常
    
    def __iter__(self):  # 返回迭代器对象本身
        return self

actions = ["wakeup", "brush", "wash", "eat", "sleep", "kill", "shot"]
custom_iter = SelfDefineIter(actions)

for x in custom_iter:
    print(x)

wakeup
brush
wash
eat
sleep


#### 迭代器协议
> 迭代器必须同时实现__next__和__iter__两个方法  
> 迭代器必须是可迭代的，即一种可迭代对象
> 所有迭代器的__iter__方法只需 return self即可
##### 作用
>- 同意通过__next__()方法获取数据，可以屏蔽底层数据的不同读取方式，简化编程。  
##### 可迭代对象
> 容器类型
>- 列表、元组、字典
>- 只有__iter__接口
>- 静态数据
>- 需要额外的迭代器支持
>- 支持多次迭代
>
> 迭代器类型
>- 文件、StringIO等
>- 同时实现__iter__、__next__接口
>- 动态数据
>- 只能一次迭代，单向
>
> So
>- 一个可迭代对象可以构建多个迭代器  
>- 一种迭代器可以应用与任意多个可迭代对象（包括本身）

In [ ]:
# 对于容器类型的可迭代对象，此时迭代器在幕后被创建，生命周期仅在for循环内
for iterable in iterables:
    pass

In [ ]:
# 对于迭代器对象，迭代器与for循环分离，迭代器的生命周期不受for循环控制
iterables = [1,2,3]
iterors = iter(iterables)
for x in iterors:
    pass


#### 应用场景
> 数据管道
>- 可以任意的嵌套连接
>- [for loop] <- [iteror] <- [iteror] <- ... <- [iteror] <- [iterable]
>- 很多个迭代器串联，形成数据数据的管道，即**数据流**
>- 在这个管道中，每一次只通过一份数据，避免了一次性加载所有数据  
>- 迭代器不仅仅是开始按顺序返回数据，开始进行**数据处理**的工作 
>- 通过迭代器获取，远离了数据存储。  
>
> 数据生成器
>- 实时生成数据，不考虑数据存储

In [1]:
from random import random

class RandomIterator:
    def __init__(self):
        return self

    def __next__(self):
        return random()

#### 附录
> 并不是所有的可迭代对象都必须有__iter__方法。
> 如果一个对象没有__iter__方法，但是定义了__getitem__方法，同样是可以迭代的。
> 因此，不能通过检查__iter__方法来判断一个对象是否是可迭代的，而是应该直接使用iter()函数，如果不可迭代，则会抛出TypeError异常。

In [3]:
class DumbList:
    def __getitem__(self, index):
        if index < 0:
            return StopIteration
        return index * 2
    
dl = DumbList()
for item in dl:
    print(item)
    if item >= 10:
        break

0
2
4
6
8
10
